# Example
of the last added functionalities in onlineSG

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import sys
sys.path.append('../')  # uncomment to add the path to the onlineSG folder 
# You have to change the path if you move this file.
# Alternatevely add the onlineSG folder to your PYTHONPATH
# e.g. On Linux add this line on .bashrc:
# export PYTHONPATH="<absolute-path-to-onlineSG>:PYTHONPATH"
# then you can use onlineSG library from everywhere

In [1]:
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.parsers as parsers
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.runner as runner
import source.players.detection as detection 
import source.util as util

from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time

In [730]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(runner)
reload(parsers)
reload(game)
#reload(environment)
reload(detection)
reload(util)

<module 'source.util' from '/home/lorenzo/Scrivania/Polimi/Thesis/code/onlineSG/source/util.py'>

In [235]:
import source.players.fabulous as fabulous

In [732]:
reload(fabulous)

<module 'source.players.fabulous' from '/home/lorenzo/Scrivania/Polimi/Thesis/code/onlineSG/source/players/fabulous.py'>

In [239]:
from math import log, sqrt
from copy import deepcopy

In [3]:
t = 1

In [233]:
t+=1
p = 0.5 / t**2
b = (sqrt(-log(p) / (t)))
print(b, t)

0.2238775040138658 231


In [1097]:
g = game.zs_game([0.4, 0.5, 0.3, 0.1, 0.7, 0.8], 1000)
d = fabulous.FABULOUS(g, 0, 1)
a = attackers.StochasticAttacker(g, 1, 1)
p1 = attackers.StochasticAttacker(g, 1, 1)
p2 = attackers.StochasticAttacker(g, 1, 1)
p3 = attackers.UnknownStochasticAttacker(g, 1, 1)
g.set_players([d], [a], [a, p1, p2, p3])

In [1098]:
e = runner.Experiment(deepcopy(g))

In [1099]:
e.run()

lt_loss 0.180151176757 is in [-0.5157335921350472141, 1.1157335921350470809]
with b 0.8157335921350471 and loss 0.3
lt_loss 0.180151176757 is in [-0.35191353338089642611, 1.2185802000475629558]
with b 0.7852468667142297 and loss 0.4333333333333333
lt_loss 0.180151176757 is in [-0.39465948221180691124, 1.0946594822118069779]
with b 0.7446594822118069 and loss 0.35
lt_loss 0.180151176757 is in [-0.32762910107965648177, 1.0876291010796563796]
with b 0.7076291010796565 and loss 0.38
lt_loss 0.180151176757 is in [-0.27540929765714111666, 1.0754092976571409945]
with b 0.6754092976571411 and loss 0.39999999999999997
lt_loss 0.180151176757 is in [-0.26173998033182632739, 1.0331685517603976798]
with b 0.647454266046112 and loss 0.3857142857142857
lt_loss 0.180151176757 is in [-0.27302682214618123613, 0.97302682214618119172]
with b 0.6230268221461812 and loss 0.35
lt_loss 0.180151176757 is in [-0.27926312945576492464, 0.9237075739002093]
with b 0.6014853516779871 and loss 0.3222222222222222
lt_l

In [ ]:
fabulous.FABULOUS.parse()

In [1100]:
e.stats

{'actual_regret': 29.864031029689794,
 'exp_regret': 31.507883957998672,
 'total_rewards': -271.5999999999986}

In [1093]:
print(a.get_best_responder().compute_strategy(), a.opt_loss())
e.agent.opt_lt, 

[0, 0, 0, 0, 1, 0] 0.345651583594


([LossTuple(loss=0.24729281430529226, br=[0, 0, 0, 0, 0, 1], attacker=<StochasticAttacker id:1 resources:1>),
  LossTuple(loss=0.33614949498421326, br=[1, 0, 0, 0, 0, 0], attacker=<StochasticAttacker id:1 resources:1>),
  LossTuple(loss=0.34565158359362325, br=[0, 0, 0, 0, 1, 0], attacker=<StochasticAttacker id:1 resources:1>)],)

In [1095]:
[h[0] for h in e.game.strategy_history]

[[0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0,

In [1096]:
len([[0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1]])

387

In [1087]:
e.run_interaction()
for lt in e.agent.opt_lt:
    print(lt)
print(a.get_best_responder().compute_strategy(), a.opt_loss())
e.agent.tau()

lt_loss 0.235200535889 is in [0.19553636942449839631, 0.53936559135981543278]
with b 0.1719146109676585 and loss 0.3674509803921569
LossTuple(loss=0.18590356728181023, br=[0, 0, 1, 0, 0, 0], attacker=<StochasticAttacker id:1 resources:1>)
LossTuple(loss=0.23520053588908357, br=[0, 1, 0, 0, 0, 0], attacker=<StochasticAttacker id:1 resources:1>)
LossTuple(loss=0.28834530472000469, br=[0, 0, 0, 0, 1, 0], attacker=<StochasticAttacker id:1 resources:1>)
[0, 0, 0, 0, 1, 0] 0.28834530472


256

In [744]:
e.agent.mock_sto.compute_strategy()

[0.6, 0.2, 0.0, 0.2, 0.0, 0.0]

In [826]:
for f in e.agent.avg_rewards:
    print(f.fixed_strategy, -e.agent.avg_rewards[f])

[0, 0, 0, 0, 1, 0] 0.27710843373493976
[1, 0, 0, 0, 0, 0] 0.37108433734939755
[0, 1, 0, 0, 0, 0] 0.37831325301204816
[0, 0, 0, 0, 0, 1] 0.3855421686746988
[0, 0, 0, 1, 0, 0] 0.43855421686746987
[0, 0, 1, 0, 0, 0] 0.46265060240963857


In [741]:
print(util.game_str(e.game))

 	 	 	 	□	 	
0.4	0.5	0.3	0.1	0.7	0.8	
△	 	 	 	 	 	
------------------------------------------------
□	 	 	 	 	 	
0.4	0.5	0.3	0.1	0.7	0.8	
△	 	 	 	 	 	
------------------------------------------------
 	 	□	 	 	 	
0.4	0.5	0.3	0.1	0.7	0.8	
 	△	 	 	 	 	
------------------------------------------------
 	 	□	 	 	 	
0.4	0.5	0.3	0.1	0.7	0.8	
△	 	 	 	 	 	


In [729]:
e.agent.mock_sto.game.history

[{0: [2], 1: [3]},
 {0: [0], 1: [2]},
 {0: [5], 1: [2]},
 {0: [5], 1: [3]},
 {0: [5], 1: [0]},
 {0: [5], 1: [3]}]

In [273]:
self = e.agent
self.norm_const = max([v[self.id] for v in self.game.values])

In [285]:
e.agent.norm_const

2

In [129]:
help(sorted)

Help on built-in function sorted in module builtins:

sorted(iterable, key=None, reverse=False)
    Return a new list containing all items from the iterable in ascending order.
    
    A custom key function can be supplied to customize the sort order, and the
    reverse flag can be set to request the result in descending order.



In [86]:
e.agent.avg_rewards

{<FixedActionDefender id:0 resources:1>: -0.75,
 <FixedActionDefender id:0 resources:1>: -0.5}

In [87]:
e.game.history

[{0: [1], 1: [0]},
 {0: [1], 1: [0]},
 {0: [0], 1: [0]},
 {0: [0], 1: [1]},
 {0: [0], 1: [0]},
 {0: [0], 1: [0]}]

In [108]:
mpi(2,3)

mpi('2.0', '3.0')

In [288]:
for p in e.game.profiles:
    if isinstance(p, attackers.StochasticAttacker):
        print(p.distribution, p.get_best_responder().compute_strategy())

[0.16128183810759994, 0.8387181618924] [0, 1]
[0.69736230169271662, 0.30263769830728338] [1, 0]
[0.130854094058935, 0.86914590594106511] [0, 1]


In [91]:
a.get_best_responder().compute_strategy()

[1, 0]

In [92]:
e.game.profiles

[<UnknownStochasticAttacker id:1 resources:1>,
 <StochasticAttacker id:1 resources:1>,
 <StochasticAttacker id:1 resources:1>,
 <StochasticAttacker id:1 resources:1>]

In [68]:
self = e.agent
for lt in self.opt_lt:
    loss = [-self.avg_rewards[k] for k
            in self.avg_rewards
            if k.last_strategy == lt.br]
    print(k.last_strategy, lt.br)

[0, 1] [1, 0]
[0, 1] [0, 1]
[0, 1] [0, 1]
[0, 1] [1, 0]


In [71]:
[k.fixed_strategy for k in self.avg_rewards]

[[0, 1], [1, 0]]

In [60]:
x = mpmath.mpi(2, 5)

In [56]:
5.1 in x

False

In [46]:
e.agent.avg_rewards

{<FixedActionDefender id:0 resources:1>: -0.3333333333333333,
 <FixedActionDefender id:0 resources:1>: -0.8333333333333334}

In [26]:
a.distribution

[0.87916302864945817, 0.12083697135054182]

In [8]:
targets_dict = {i:[] for i in range(2,13)}
T = 15 #np.random.randint(2,10)
time_horizon =1000
while sum([len(targets_dict[x]) for x in range(2,13)]) < 9:
    targets = [round(x,3) for x in util.gen_norm_targets(T)]
    values = tuple((v, v) for v in targets)
    g = game.Game(values, time_horizon)
    g.attackers = [1]
    g.defenders = [0]
    s = util.support(g)
    if len(targets_dict[len(s)])==0:
        targets_dict[len(s)].append(s)

In [9]:
targets_dict

{2: [[0.94, 0.709]],
 3: [[0.899, 0.872, 0.872]],
 4: [[0.706, 0.796, 0.744, 0.821]],
 5: [[0.659, 0.763, 0.858, 0.681, 0.73]],
 6: [[0.605, 0.986, 0.931, 0.629, 0.703, 0.594]],
 7: [[0.715, 0.676, 0.675, 0.88, 0.925, 0.656, 0.89]],
 8: [[0.636, 0.674, 0.958, 0.742, 0.736, 0.764, 0.676, 0.687]],
 9: [[0.989, 0.873, 0.74, 0.973, 0.73, 0.894, 0.778, 0.769, 0.741]],
 10: [[0.818, 0.987, 0.864, 0.937, 0.924, 0.872, 0.914, 0.991, 0.81, 0.921]],
 11: [],
 12: []}

In [5]:
time_horizon =10

In [10]:
T = 2
P = 1
targets = translate(targets_dict[T][0])
distributions = []
for i in range(P):
    distributions.append(tuple(gen_distr(T)))
values = tuple((v, v) for v in targets)
g = game.Game(values, time_horizon)
#print(T, targets)
profiles = [attackers.StackelbergAttacker(g, 1),
            attackers.UnknownStochasticAttacker(g, 1)]
for d in distributions[1:]:
    #print(d)
    profiles.append(attackers.StochasticAttacker(g, 1, 1, *d))
#agent = defenders.BR_MAB(g, 0, 1)
agent = detection.HOLMES(g, 0, 1, 2)
attacker = attackers.StochasticAttacker(g, 1, 1, *distributions[0])
g.set_players([agent], [attacker], profiles)

In [7]:
from copy import deepcopy

In [105]:
c = runner.Configuration(deepcopy(g), "trial", "trial", False)

In [18]:
c2 = runner.Configuration(deepcopy(g), "trial", "trial2", False)

In [9]:
rm -rf trial/trial

In [8]:
e =runner.Experiment(deepcopy(g))

In [9]:
start_time = time.time()
e.run_interaction()
print(time.time() - start_time)
print(game_str(e.game, lenght=7))
print(e.agent.belief)
print(e.game.strategy_history[-1])


0.5140483379364014
 	□	
0.298	0.413	
△	 	
{<StackelbergAttacker id:1 resources:1>: 0.6000000000000001, <UnknownStochasticAttacker id:1 resources:1>: 0.4}
{0: (0.41812798874824186, 0.5818720112517581), 1: [0.74074067539405486, 0.25925932460594497]}


In [12]:
e.game.values

((0.29799999999999993, 0.29799999999999993),
 (0.4129999999999999, 0.4129999999999999))

In [13]:
0.29799999999999993 + 0.4129999999999999

0.7109999999999999

In [11]:
e.agent.tree.print_tree(end=2)

 state  b: [0.5, 0.5] r: 0 p: 1
 branches:

 (0, 1)
	 exp_regret  0.10512025
	 branches:

	 (1, 0)
		 state  b: [0.6666666666666666, 0.3333333333333333] r: -0.29799999999999993 p: 0.75
		 branches:

		 (1, 0)
			 exp_regret  0.22248347398030935
			 branches:

			 (0, 1)
				 state  b: [0.8571428571428572, 0.14285714285714288] r: -0.7109999999999999 p: 0.5833333333333333
			 (0, 0)
				 state  b: [0.0, 1.0] r: -0.29799999999999993 p: 0.16666666666666666
		 (0.41812798874824186, 0.5818720112517581)
			 exp_regret  0.11155595182841069
			 branches:

			 (0, 1)
				 state  b: [0.0, 1.0] r: -0.7109999999999999 p: 0.034843999062353484
			 (1, 0)
				 state  b: [0.75, 0.25] r: -0.5959999999999999 p: 0.38791467416783876
			 (0, 0)
				 state  b: [0.75, 0.25] r: -0.29799999999999993 p: 0.27875199249882787
			 (1, 1)
				 state  b: [0.0, 1.0] r: -0.29799999999999993 p: 0.048489334270979845
		 (0.42012798874824187, 0.5798720112517581)
			 exp_regret  0.11158295182841065
			 branches:

			 (0, 1)


In [96]:
util.plot_conf2("exp_regret", [c], "trial")

In [164]:
rm -rf trial

In [165]:
b = runner.Batch("/home/lorenzo/Scrivania/Experiments/UnknownStochastic_Stackelberg2/t4/p1/br_mab1-1_br_expert1-1_b2bw2w1_holmes1-1.csv",
                "trial")

In [166]:
b.parse_batch()

In [78]:
b.configurations[0]

<Configuration game:1000,0.778,0.893,0.545,0.747,br_mab1-1,sto1-0.471429133561-0.00602317029063-0.096473268935-0.426074427213,sta1,unk_stochastic_attacker1,sto1-0.0346193189847-0.0966672214518-0.379762168635-0.488951290929 experiments:[] stats:{}>

In [26]:
from copy import deepcopy

In [176]:
g = deepcopy(b.configurations[0].game)

In [178]:
c = runner.Configuration(g, ".", "ciao", print_results=False)

In [186]:
c.run()

In [190]:
util.plot_conf2("exp_regret", [c], ".")

In [189]:
ls 

1_1.csv                           Fighting Stochastic-2.0.ipynb
B2BW2W_expert_mab1.csv            Fighting Stochastic.ipynb
B2BW2W_expert_mab2.csv            gurobi.log
batch/                            help.txt
batch1.csv                        Il carcere sovraffollato.ipynb
batch.csv                         Linear Programming Examples.ipynb
ciao/                             Matplotlib trials.ipynb
ciao.txt                          plot
Compare Algorithms.ipynb          Presentation.ipynb
Compare configurations.ipynb      prova.csv
Compute Losses.ipynb              Results.ipynb
concurrent/                       Run Batch.ipynb
concurrent1/                      Run Configuration.ipynb
difficulties/                     Runner.ipynb
Example-Copy1.ipynb               Stackerlberg Best Response.ipynb
Example-Copy2.ipynb               status.txt
Example-Copy3.ipynb               sto_sta/
Example.ipynb*                    sto_sta1/
Experiments-1.ipynb               trial/
Experiments-Prepar

In [185]:
c.experiments

[]

In [170]:
e = runner.Experiment(g, 45333)

In [29]:
for i in range(100):
    e.run_interaction()

uniform
1.404613567831535
1.0884789134632844
-0.44447984592909195
1.4893842466621894
-0.29647691163777634
-0.2604539637001497
0.17516859941865592
0.3318221104763115
0.06682730874152987
-0.0513341196736391
-0.08178556329469044
-0.22740471991963096
-0.27344128420270875
-0.5006459158220038
-0.6259784330242586
-0.31903861462725674
-0.17474240944648484
-0.5088426484985966
-0.47674104393793637
-0.19346136714711515
-0.33971333195567754
-0.36512899988274444
-0.4468543745262948
-0.33405759688170267
-0.19913266106844552
-0.4674503026311515
-0.5518659186165786
-0.1885813497705484
-0.3381673082935635
-0.50749260136533
-0.4839845385228221
-0.3150854952984752
-0.21075277462361858
-0.6065548297629414
-0.5392268694015836
-0.33538294761434195
-0.26859164362795684
-0.6568714940915499
-0.6059925210789266
-0.32905070331221065
-0.23226561740507734
-0.6292372204394215
-0.5952625177376483
-0.3674699103940948
-0.18038606664889745
-0.5889775299489857
-0.6440683909789091
-0.42413021862786343
-0.3480077738337007

In [171]:
e.run()

In [172]:
e.stats

{'actual_regret': 35.960780232780778,
 'exp_regret': 22.677175004483665,
 'total_rewards': -412.19500000000437}

In [104]:
e.game.players[1].opt_loss()

0.3762342197672186

In [141]:
list(e.agent.arms.items())[2][1].mock_sto is e.game.profiles[2]

False

In [174]:
e.agent.arms[0].mock_sto is e.game.profiles[0]

False

In [175]:
e.game.profiles[0]

<UnknownStochasticAttacker id:1 resources:1>

In [45]:
list(e.agent.arms.items())[1][1].embedded

False

In [450]:
d1 = list(e.agent.arms.items())[0][1].avg_rewards
for k in d1:
    print(k.last_strategy, d1[k])

[0, 0, 0, 1] -0.42754807692307695
[0, 0, 1, 0] -0.6911826923076922
[1, 0, 0, 0] -0.3770288461538462
[0, 1, 0, 0] -0.7350000000000001


In [489]:
list(e.agent.arms.items())[0][1].__dict__

{'_finalized': True,
 'algorithm': <ExpAlgorithm.fpls: 3>,
 'arms': (<FixedActionDefender id:0 resources:1>,
  <FixedActionDefender id:0 resources:1>,
  <FixedActionDefender id:0 resources:1>,
  <FixedActionDefender id:0 resources:1>),
 'avg_rewards': {<FixedActionDefender id:0 resources:1>: -0.42754807692307695,
  <FixedActionDefender id:0 resources:1>: -0.6911826923076922,
  <FixedActionDefender id:0 resources:1>: -0.3770288461538462,
  <FixedActionDefender id:0 resources:1>: -0.7350000000000001},
 'embedded': False,
 'feedbacks': [],
 'game': <Game values:[[0.778, 0.778], [0.893, 0.893], [0.545, 0.545], [0.747, 0.747]] players{0: <HOLMES id:0 resources:1>, 1: <StochasticAttacker id:1 resources:1>} time_horizon:1000>,
 'id': 0,
 'last_strategy': [1, 0, 0, 0],
 'learning': <Learning.EXPERT: 2>,
 'mock_sto': <UnknownStochasticAttacker id:1 resources:1>,
 'norm_const': 0.893,
 'resources': 1,
 'sel_arm': <FixedActionDefender id:0 resources:1>}

In [421]:
e.agent.learning

<Learning.MAB: 1>

In [382]:
e.run()

In [383]:
e.stats

{'actual_regret': 25.536780232781361,
 'exp_regret': 41.187497530224164,
 'total_rewards': -401.77100000000274}

In [332]:
e.run_interaction()
print(e.game.strategy_history[-1])
print(e.agent.belief)
#print(util.game_str(e.game, lenght=7))
print(e.game.history[-1], e.agent.tau())

{0: (0, 0, 0, 1), 1: [0.471429133561, 0.00602317029063, 0.096473268935, 0.426074427213]}
{<UnknownStochasticAttacker id:1 resources:1>: 1.0, <StochasticAttacker id:1 resources:1>: 2.800258122799861e-48, <StackelbergAttacker id:1 resources:1>: 0.0}
{0: [3], 1: [3]} 111


In [373]:
list(e.agent.arms.items())[0][1].compute_strategy()

[0, 1, 0, 0]

In [374]:
e.agent.profiles[1].last_strategy

[0.4434782608695652,
 0.017391304347826087,
 0.08695652173913043,
 0.45217391304347826]

In [312]:
e.game.players[1].distribution

[0.471429133561, 0.00602317029063, 0.096473268935, 0.426074427213]

In [143]:
e.agent.get_br_strategies(e.agent.arms)

[(0, 0, 0, 1), (0, 0, 1, 0)]

In [153]:
[tuple(e.agent.arms[k].compute_strategy()) for k in e.agent.profiles if k.__class__.name != attackers.StackelbergAttacker.name]

[(0, 0, 0, 1), (0, 1, 0, 0)]

In [117]:
e.agent.profiles

[<StochasticAttacker id:1 resources:1>,
 <UnknownStochasticAttacker id:1 resources:1>,
 <StackelbergAttacker id:1 resources:1>]

In [ ]:
e.agent.tree.branches[(0,0,0,1)].branches[()]

In [ ]:
p 0.32900284734860696
p 0.11090171664439692
p 0.11500577150033305
p 0.44508966450667015

In [ ]:
0.3, 0.1, 0.1, 0.5
0.0346193189847, 0.0966672214518, 0.379762168635, 0.488951290929
0.471429133561, 0.00602317029063, 0.096473268935, 0.426074427213

In [97]:
e.agent.tree.print_tree(end=1)

 state  b: [0.014497096810704484, 0.0, 0.9855029031892955] r: 0 p: 1
 branches:

 (0, 0, 0, 1)
	 exp_regret  -2.3314683517128287e-15
	 branches:

	 (3, 0)
		 state  b: [0.0015254567639351483, 0.0, 0.9984745432360649] r: -0.778 p: 0.32900284734860696
	 (3, 2)
		 state  b: [0.047871066398860976, 0.0, 0.952128933601139] r: -0.545 p: 0.11500577150033305
	 (3, 1)
		 state  b: [0.012636360465925721, 0.0, 0.9873636395340744] r: -0.893 p: 0.11090171664439692
	 (3, 3)
		 state  b: [0.015925721861399053, 0.0, 0.9840742781386009] r: 0.0 p: 0.44508966450667015
 (0, 0, 1, 0)
	 exp_regret  0.2698038339187988
	 branches:

	 (2, 0)
		 state  b: [0.0015254567639351483, 0.0, 0.9984745432360649] r: -0.778 p: 0.32900284734860696
	 (2, 3)
		 state  b: [0.015925721861399053, 0.0, 0.9840742781386009] r: -0.747 p: 0.44508966450667015
	 (2, 1)
		 state  b: [0.012636360465925721, 0.0, 0.9873636395340744] r: -0.893 p: 0.11090171664439692
	 (2, 2)
		 state  b: [0.047871066398860976, 0.0, 0.952128933601139] r: 0.0

In [39]:
player.Attacker(g,1).exp_loss({0: (0,0,0, 1), 1: (0.32900284734860696, 0.11090171664439692, 0.11500577150033305, 0.44508966450667015)})

0.4176775936683442

In [40]:
player.Attacker(g,1).exp_loss({0: (1,0,0,0), 1: (0.32900284734860696, 0.11090171664439692, 0.11500577150033305, 0.44508966450667015)})

0.4941953578176106

In [71]:
e.game.players[1].get_best_responder().compute_strategy()

[1, 0, 0, 0]

In [69]:
list(e.agent.arms.items())[1][1].tau

{<FixedActionDefender id:0 resources:1>: -0.6614,
 <FixedActionDefender id:0 resources:1>: -0.6961999999999999,
 <FixedActionDefender id:0 resources:1>: -0.5173,
 <FixedActionDefender id:0 resources:1>: -0.3772}

In [71]:
e.game.profiles[1].get_best_responder().tau

0

In [68]:

d = e.game.profiles[0].get_best_responder().avg_rewards
for k in d:
    print(k.fixed_strategy, d[k])

[0, 0, 0, 1] -0.4191111111111111
[0, 1, 0, 0] -0.6518888888888889
[0, 0, 1, 0] -0.6905555555555555
[1, 0, 0, 0] -0.49177777777777776


In [264]:
e.game.players[1].exp_loss({0: (1,0,0,0), 1:None})

0.3762342197672186

In [263]:
g.players[1].exp_loss({0: (0,0,0,1), 1:None})

0.42472848854956563

In [40]:
e.agent.tree.branches[(0,0,0,1)]

exp_regret: -2.3314683517128287e-15branches: {(3, 0): state: b: [0.9984745432360649, 0.0015254567639351483, 0.0] r: -0.778 p: 0.32900284734860696branches: {}, (3, 2): state: b: [0.952128933601139, 0.047871066398860976, 0.0] r: -0.545 p: 0.11500577150033305branches: {}, (3, 1): state: b: [0.9873636395340744, 0.012636360465925721, 0.0] r: -0.893 p: 0.11090171664439692branches: {}, (3, 3): state: b: [0.9840742781386009, 0.015925721861399053, 0.0] r: 0.0 p: 0.44508966450667015branches: {}}

In [27]:
g.profiles[0].exp_loss({0: (1,0,0,0), 1:None})

0.6585408249864955

In [30]:
g.profiles[0].exp_loss({0: (0,0,0,1), 1:None})

0.320228040832629

In [29]:
e.game.players[1].exp_loss({0: (1,0,0,0), 1:None})

0.3762342197672186

In [88]:
e.game.players[1].exp_loss({0: (0,0,0,1), 1:None})

0.42472848854956563

In [39]:
g.players[1].distribution

[0.471429133561, 0.00602317029063, 0.096473268935, 0.426074427213]

In [43]:
g.profiles[0].compute_strategy()

[0.3, 0.1, 0.1, 0.5]

In [45]:
g.profiles[2].distribution

[0.0346193189847, 0.0966672214518, 0.379762168635, 0.488951290929]

In [33]:
0.4748799881664601 * 0.1728292542209529

0.08207315419926424